In [ ]:
!pip install langchain langchain-groq langchain-community langchain-text-splitters sentence-transformers faiss-cpu pypdf python-dotenv

In [1]:

import streamlit as st
import os
import tempfile
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

c:\Users\PC\Desktop\VS code File\Rag_groq\rag_croq_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [ ]:

# Load PDF
loader = PyPDFLoader('C:/Users/PC/Desktop/VS code File/RAG_GROQ/RL exam 25-26.pdf')
docs = loader.load()

#if you want to use the web scrapping 
loader = WebBaseLoader(website_url )
docs = loader.load()
            
            
# Split text
splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000, 
                chunk_overlap=200
            )
chunks = splitter.split_documents(docs)
    
            
# Create embeddings
embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )

In [6]:
# Let's define the query, since we are going to use it multiple times.
query = "Give me the main comptetence of mohamed"



db = FAISS.from_documents(chunks, embeddings)

In [7]:
retrieved_docs = db.similarity_search(query, k=3)
context = "\n\n".join([d.page_content for d in retrieved_docs])

 
# initialize the retriever
retriever = db.as_retriever(
search_type="mmr", #similarity
search_kwargs={'k': 4}
)


In [8]:
llm = ChatGroq(
                        model="llama-3.3-70b-versatile",  
                        temperature=0.3,
                        max_tokens=512
                    )

In [10]:
m=llm.invoke("resume this file in 2 ligne")

In [11]:
m

AIMessage(content="There is no file to resume. Please provide the file or text you would like me to summarize, and I'll be happy to help. \n\nIf you provide the text, I can summarize it for you in 2 lines.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 42, 'total_tokens': 89, 'completion_time': 0.130330821, 'completion_tokens_details': None, 'prompt_time': 0.009367744, 'prompt_tokens_details': None, 'queue_time': 0.295427899, 'total_time': 0.139698565}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3272ea2d91', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b0d9b-66a0-7983-a2b4-af78e6196342-0', usage_metadata={'input_tokens': 42, 'output_tokens': 47, 'total_tokens': 89})

In [13]:
# Create prompt
prompt = ChatPromptTemplate.from_template("""
Réponds à la question en te basant uniquement sur le contexte fourni.
Si tu ne trouves pas la réponse dans le contexte, dis "Je n'ai pas cette information dans le document."

Contexte:
{context}

Question: {question}

Réponse:""")

In [18]:
from langchain_core.runnables import RunnablePassthrough              

output_parser = StrOutputParser()

chain = (
{"context": retriever, "question": RunnablePassthrough()}
| prompt
| llm
| output_parser
)


In [ ]:
response = chain.invoke("donne moi le resume de ca")
response

"Le projet consiste à concevoir et à entraîner un modèle de langage léger (1-10 millions de paramètres) capable de réaliser correctement des additions et des soustractions de petits entiers (par exemple, des nombres allant jusqu'à 3-4 chiffres). Le modèle doit d'abord être pré-entraîné sur un objectif de token suivant simple, puis affiné à l'aide de l'apprentissage par renforcement pour améliorer la précision mathématique.\n\nLes étapes clés incluent :\n\n1. Création d'un jeu de données synthétique d'expressions arithmétiques avec des résultats corrects.\n2. Utilisation d'un modèle de transformateur très petit (2-4 couches) avec une taille d'incrustation de 64-128, 2-4 têtes d'attention et une longueur de séquence de 10-20 jetons.\n3. Entraînement du modèle à reproduire les sorties token par token, puis affinage à l'aide de l'apprentissage par renforcement pour encourager les réponses mathématiques correctes.\n4. Évaluation des métriques telles que la précision sur l'addition et la sou